In [ ]:
%reset

In [ ]:
# Change wd to the path of causal_inference folder as the package has no setup



In [ ]:
import os, sys

import pandas as pd
import numpy as np
import swifter

import pacmagic
import pacmagic_icu

from importlib import reload
from data_warehouse_utils.dataloader import DataLoader

os.chdir('/home/adam/files/causal_inference')
os.getcwd()

from causal_inference.experiment_generator.initialize_experiment import initialize_experiment
from causal_inference.experiment_generator.initialize_experiment import add_parameter
from causal_inference.experiment_generator.initialize_experiment import _load_data_patients

In [ ]:
# Reloads packages

reload(sys.modules['causal_inference'])
reload(sys.modules['causal_inference.experiment_generator'])
reload(sys.modules['causal_inference.experiment_generator.initialize_experiment'])


from causal_inference.experiment_generator.initialize_experiment import initialize_experiment

In [ ]:
dl = DataLoader()

df, generator = initialize_experiment(dl = dl,
                           n_of_patients = 10,
                           min_length_of_intubation = 24,
                           length_of_time_window_hours = 4)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
generator_test = generator[0:5]

In [ ]:
BMI = ['body_mass_index']

In [ ]:
LAB_VALUES = ['c_reactive_protein',
              'leukocytes',
              'procalcitonin',
              'hemoglobin',
              'creatinine',
              'ureum',
              'chloride',# is this instead of chlorine?
              'sodium',
              'calcium',
              'phosphate',
              'glucose',
              'activated_partial_thromboplastin_time',
              'prothrombin_time_inr',
              'd_dimer',
              'fibrinogen',
              'alanine_transaminase',
              'aspartate_transaminase'
              'lactate_dehydrogenase',
              'albumin',
              'creatine_kinase',
              'gamma_glutamyl_transferase',
              'alkaline_phosphatase',
              'bilirubin_direct']

In [ ]:
BLOOD_GAS = ['pco2', 'po2', 'bicarbonate', 'lactate' ]


In [ ]:
VITAL_SIGNS = ['heart_rate',
               'arterial_blood_pressure_mean',
               'arterial_blood_pressure_diastolic',
               'arterial_blood_pressure_systolic']

In [ ]:
VENTILATOR_VALUES = ['peep',
                     'fio2',
                     'pressure_above_peep',
                     'tidal_volume',
                     'inspiratory_expiratory_ratio',
                     'respiratory_rate' #this doesn't work and the name is correct
                     'lung_compliance',
                     'driving_pressure',
                     'plateau_pressure',
                     'peak_pressure']

In [ ]:
values = pd.DataFrame([])

for idx, row in enumerate(generator_test):

    patient_id = [generator[idx][0]]
    patient_df = generator[idx][1]

    parameter_value = patient_df[['time_window_start', 'time_window_end']].\
        swifter.\
        apply(lambda row: dl.get_single_timestamp(patients = patient_id,
                                                  parameters = LAB_VALUES,
                                                  columns = ['pacmed_name',
                                                             'numerical_value'],
                                                  from_timestamp = row.time_window_start,
                                                  to_timestamp = row.time_window_end).\
              groupby('pacmed_name').\
              mean(numeric_only = False).
              T,
              axis = 1)

    parameter_value = pd.concat([parameter_value[idx] for idx in range(len(parameter_value))])
    parameter_value.reset_index(inplace = True, drop=True)
    values = pd.concat([values, parameter_value])

In [ ]:
values.info()

In [ ]:
values


In [ ]:
df_test = df.sample(10)

In [ ]:
PARAMETER_NAME = ['fio2', 'peep', 'pao2']

values = pd.DataFrame([])

for idx, row in df_test.iterows():

    patient_df = generator[idx][1]

    parameter_value = patient_df[['hash_patient_id', 'time_window_start', 'time_window_end']].\
        swifter.\
        apply(lambda row: dl.get_single_timestamp(patients = [row.hash_patient_id],
                                                  parameters = PARAMETER_NAME,
                                                  columns = ['pacmed_name',
                                                             'numerical_value'],
                                                  from_timestamp = row.time_window_start,
                                                  to_timestamp = row.time_window_end).\
              groupby('pacmed_name').\
              mean(numeric_only = False).
              T,
              axis = 1)

    parameter_value = pd.concat([parameter_value[idx] for idx in range(len(parameter_value))])
    parameter_value.reset_index(inplace = True, drop=True)
    values = pd.concat([values, parameter_value])

In [ ]:
values.head()

In [ ]:
dl.get_single_timestamp(patients = [df.hash_patient_id.iloc[4]],
                        parameters = ['spo2'],
                        columns = ['pacmed_name', 'pacmed_subname', 'numerical_value', 'effective_timestamp'],
                        from_timestamp = df.time_window_start.iloc[1],
                        to_timestamp = df.time_window_end.iloc[500])





In [ ]:
measurements_range.head()

In [ ]:
patients = _load_data_patients(dl, min_length_of_stay_hours = 96)

In [ ]:
patients.head()

In [ ]:
patients['bmi_timestamp'] = patients.apply(lambda row: dl.get_single_timestamp(
    patients = [row.hash_patient_id],
    parameters = ['body_mass_index'],
    columns = ['numerical_value'],
    from_timestamp = row.start_timestamp,
    to_timestamp = row.end_timestamp).first(),
              axis = 1)

In [ ]:
dl.get_single_timestamp(patients = [patients.hash_patient_id.first()],
    parameters = ['body_mass_index'],
    columns = ['numerical_value'],
    from_timestamp = row.start_timestamp,
    to_timestamp = row.end_timestamp)

In [ ]:
patients.hash_patient_id

In [ ]:
patients.head()

In [ ]:
df_proning.info()

In [ ]:
df_proning.head(10)

In [ ]:
#### Procesing proned patients

df_proning = dl.get_range_measurements(parameters= ['position'],
                                       sub_parameters=['position_body'],
                                       columns=['hash_patient_id',
                                                'start_timestamp',
                                                'end_timestamp',
                                                'effective_value',
                                                'is_correct_unit_yn',
                                                'pacmed_origin_hospital',
                                                'fake_admission_id'
                                                ]
                                       )

df_proning.sort_values(by = ['hash_patient_id', 'start_timestamp'],
                       ascending = True,
                       inplace = True)
df_proning.reset_index(drop=True, inplace=True)

In [ ]:
df_proning.info()

In [ ]:
df_proning.head(5)

In [ ]:
df_proned = df_proning[df_proning.effective_value == 'prone']

In [ ]:
df_proned.info()

In [ ]:
df_proned.head()

In [ ]:
df_proned['duration'] = df_proned.end_timestamp - df_proned.start_timestamp
df_proned['duration'] = df_proned.duration.astype('timedelta64[h]')

In [ ]:
df_proned.describe()

In [ ]:
proning_session_id = 0
df_proning['session_end'] = 0
df_proning['proning_session_id'] = 0

for index, row in df_proning.iterrows():
    df_proning.proning_session_id[index] = proning_session_id
    df_proning['session_end'] = df_proning.start_timestamp[index+1] #use diff instead
    if (df_proning.effective_value[index] != df_proning.effective_value[index + 1]):
        proning_session_id = proning_session_id + 1

start with taking hash_id and blood gas measurement as point

function for every patient blood gas measurement says if it was proned afterwards

function that for each point checks preconditions

patients -> treated -> proning sessions -> included

patients -> never treated -> then look at points with blood gas -> if inclusion yes then data point

In [ ]:
df_proning.groupby('proning_session_id')['end_timestamp'].first()

In [ ]:
df.groupby('proning_session_id').agg({'B': ['min', 'max'], 'C': 'sum'})

In [ ]:
df_proned

In [ ]:
df_proned.proning_session_id.nunique()

In [ ]:

df_proning.info()